In [1]:
%run -m ipy_startup
%run -m ipy_logging
%matplotlib inline
from musc_genomics import data
pd.set_option('max_info_rows', 10000)

In [2]:
d_raw = data.load('prepared', 'modeling_data_merge_raw')

2016-07-17 07:42:24,221:DEBUG:research.project.manager: Loading saved data from location "/Users/eczech/data/research/musc_genomics/prepared/modeling_data_merge_raw.pkl"


In [4]:
d_raw.filter(regex='^RES:|^CL:').info()

<class 'pandas.core.frame.DataFrame'>
Index: 418 entries, 22RV1 to ZR7530
Data columns (total 12 columns):
CL:CANCER_TYPE        418 non-null object
CL:HISTOLOGY          418 non-null object
CL:PRIMARY_SITE       418 non-null object
CL:GENDER             418 non-null object
RES:CANCERTYPE        418 non-null object
RES:TISSUE            418 non-null object
RES:VAL:ABT-263       413 non-null float64
RES:VAL:AG-014699     411 non-null float64
RES:VAL:NUTLIN-3A     413 non-null float64
RES:VAL:PD-0332991    391 non-null float64
RES:VAL:PLX4720       413 non-null float64
RES:VAL:SB590885      398 non-null float64
dtypes: float64(6), object(6)
memory usage: 42.5+ KB


In [6]:
d_raw.filter(regex='^RES:|^CL:').head()

FEATURE,CL:CANCER_TYPE,CL:HISTOLOGY,CL:PRIMARY_SITE,CL:GENDER,RES:CANCERTYPE,RES:TISSUE,RES:VAL:ABT-263,RES:VAL:AG-014699,RES:VAL:NUTLIN-3A,RES:VAL:PD-0332991,RES:VAL:PLX4720,RES:VAL:SB590885
TUMOR_ID,,,,,,,,,,,,
22RV1,solid,carcinoma,prostate,male,urogenital_system,prostate,5.010086,2.676501,2.549906,0.517856,6.697773,4.639378
2313287,solid,carcinoma,stomach,male,digestive_system,stomach,3.649499,6.000943,3.039341,3.912658,6.753041,6.373648
5637,solid,carcinoma,urinary_tract,male,urogenital_system,bladder,1.283993,2.897053,6.412843,5.887130,2.998122,5.108025
639V,solid,carcinoma,urinary_tract,male,urogenital_system,bladder,3.378504,5.784257,3.779543,3.054728,6.816524,6.274532
647V,solid,carcinoma,urinary_tract,male,urogenital_system,bladder,4.510450,3.973549,6.739784,5.881208,6.842532,6.625275


In [24]:
d_raw['CL:HISTOLOGY'].value_counts()

carcinoma                                                     266
lymphoid_neoplasm                                              37
malignant_melanoma                                             30
glioma                                                         25
haematopoietic_neoplasm                                        21
neuroblastoma                                                   9
mesothelioma                                                    6
unknown                                                         4
osteosarcoma                                                    4
rhabdomyosarcoma                                                3
rhabdoid_tumour                                                 2
ewings_sarcoma-peripheral_primitive_neuroectodermal_tumour      2
primitive_neuroectodermal_tumour-medulloblastoma                2
sarcoma                                                         2
carcinoid-endocrine_tumour                                      1
fibrosarco

In [37]:
c_idx = ['TUMOR_ID', 'RES:CANCERTYPE', 'RES:TISSUE']
d = d_raw.reset_index()

# Ensure that all metadata values are currently non-null before pushing them
# into an index
assert np.all(d[c_idx].notnull())

d = d.set_index(c_idx)

d.iloc[:5, :5]

,,FEATURE,CN:A1BG,CN:A1BG-AS1,CN:A1CF,CN:A2M,CN:A2ML1
TUMOR_ID,RES:CANCERTYPE,RES:TISSUE,,,,,
22RV1,urogenital_system,prostate,-0.08084,-0.08084,-0.00250,0.4486,0.4486
2313287,digestive_system,stomach,-0.03310,-0.03310,-0.05348,-0.1917,-0.1917
5637,urogenital_system,bladder,-0.03514,-0.03514,-0.49350,-0.0657,-0.0657
639V,urogenital_system,bladder,0.08280,0.08280,0.05900,0.3126,0.3126
647V,urogenital_system,bladder,0.29130,0.29130,-0.32300,0.1603,0.1603


## Minimally Encoded Feature Set

In [62]:
from sklearn.preprocessing import LabelEncoder

d_encode_min = d.copy()
encoders = {}

# For each of the following non-numeric values, apply a label encoder
# and save that encoder in a map to eventually be attached to resulting data frame
c_encode = ['CL:PRIMARY_SITE', 'CL:CANCER_TYPE', 'CL:HISTOLOGY', 'CL:GENDER']
for c in c_encode:
    assert np.all(d_encode_min[c].notnull()), 'Categorical variable "{}" has null values'.format(c)
    encoders[c] = LabelEncoder().fit(d_encode_min[c].values)
    d_encode_min[c] = encoders[c].transform(d_encode_min[c]).astype(np.float64)
d_encode_min.label_encoders = encoders

# Ensure that all features have been encoded in some numeric form at this point
assert np.all(d_encode_min.dtypes == np.float64), 'Some features are still not floating point values'

In [73]:
d_encode_min.filter(regex='^CL:').describe().T

,count,mean,std,min,25%,50%,75%,max
FEATURE,,,,,,,,
CL:CANCER_TYPE,418.0,0.870813,0.363252,0.0,1.0,1.0,1.0,2.0
CL:HISTOLOGY,418.0,3.476077,3.923202,0.0,1.0,1.0,5.0,18.0
CL:PRIMARY_SITE,418.0,9.215311,5.874920,0.0,5.0,9.0,12.0,22.0
CL:GENDER,418.0,0.703349,0.633242,0.0,0.0,1.0,1.0,2.0


In [66]:
data.save('features', 'encode_minimal', d_encode_min)

2016-07-17 08:26:50,657:DEBUG:research.project.manager: Saving data to location "/Users/eczech/data/research/musc_genomics/features/encode_minimal.pkl"


'/Users/eczech/data/research/musc_genomics/features/encode_minimal.pkl'

## Maximally Encoded Feature Set

In [78]:
d_encode_max = d.copy()

# For each of the following non-numeric values, apply dummy encoding
c_encode = ['CL:PRIMARY_SITE', 'CL:CANCER_TYPE', 'CL:HISTOLOGY', 'CL:GENDER']
for c in c_encode:
    assert np.all(d_encode_max[c].notnull()), 'Categorical variable "{}" has null values'.format(c)
    
    d_encode_max = pd.concat([
        d_encode_max.drop(c, axis=1),
        pd.get_dummies(d_encode_max[c], prefix_sep=':', prefix=c)
    ], axis=1)

    # Remove "unknown" values?
    #     c_ref = c + ':unknown'
    #     if c_ref in d_encode_max:
    #         d_encode_max = d_encode_max.drop(c_ref, axis=1)
    
# Ensure that all features have been encoded in some numeric form at this point
assert np.all(d_encode_max.dtypes == np.float64), 'Some features are still not floating point values'

In [80]:
d_encode_max.iloc[:3, :5]

,,FEATURE,CN:A1BG,CN:A1BG-AS1,CN:A1CF,CN:A2M,CN:A2ML1
TUMOR_ID,RES:CANCERTYPE,RES:TISSUE,,,,,
22RV1,urogenital_system,prostate,-0.08084,-0.08084,-0.00250,0.4486,0.4486
2313287,digestive_system,stomach,-0.03310,-0.03310,-0.05348,-0.1917,-0.1917
5637,urogenital_system,bladder,-0.03514,-0.03514,-0.49350,-0.0657,-0.0657


In [79]:
d_encode_max.filter(regex='^CL:').describe().T

,count,mean,std,min,25%,50%,75%,max
FEATURE,,,,,,,,
CL:PRIMARY_SITE:autonomic_ganglia,418.0,0.021531,0.145320,0.0,0.0,0.0,0.0,1.0
CL:PRIMARY_SITE:bone,418.0,0.014354,0.119088,0.0,0.0,0.0,0.0,1.0
CL:PRIMARY_SITE:breast,418.0,0.081340,0.273684,0.0,0.0,0.0,0.0,1.0
CL:PRIMARY_SITE:central_nervous_system,418.0,0.064593,0.246101,0.0,0.0,0.0,0.0,1.0
CL:PRIMARY_SITE:endometrium,418.0,0.021531,0.145320,0.0,0.0,0.0,0.0,1.0
CL:PRIMARY_SITE:haematopoietic_and_lymphoid_tissue,418.0,0.141148,0.348592,0.0,0.0,0.0,0.0,1.0
CL:PRIMARY_SITE:kidney,418.0,0.023923,0.152994,0.0,0.0,0.0,0.0,1.0
CL:PRIMARY_SITE:large_intestine,418.0,0.062201,0.241810,0.0,0.0,0.0,0.0,1.0
CL:PRIMARY_SITE:liver,418.0,0.014354,0.119088,0.0,0.0,0.0,0.0,1.0


In [81]:
data.save('features', 'encode_maximal', d_encode_max)

2016-07-17 08:31:59,596:DEBUG:research.project.manager: Saving data to location "/Users/eczech/data/research/musc_genomics/features/encode_maximal.pkl"


'/Users/eczech/data/research/musc_genomics/features/encode_maximal.pkl'